# 英文文本分类——电影评论情感判别
博客链接地址：https://asialee.blog.csdn.net/article/details/86497085

In [1]:
import os
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import nltk
from nltk.corpus import stopwords



In [2]:
#用pandas读入训练数据
#datafile=os.path.join('f:\\NLP学习路径总结\data\english_data','labeledTrainData.tsv')
datafile='E:\\NLP学习路径总结\\data\\english_data\\labeledTrainData.tsv'
print(datafile)
df=pd.read_csv("labeledTrainData.tsv",sep='\t',escapechar='\\')
print('Number of reviews:{}'.format(len(df)))
df.head()

E:\NLP学习路径总结\data\english_data\labeledTrainData.tsv
Number of reviews:25000


,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"""The Classic War of the Worlds"" by Timothy Hin..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [3]:
# 构建停用词列表数据
#words_nostop=[w for w in words if w not in stopwords.words('english')]
stopwords={}.fromkeys([line.rstrip() for line in open('F:\\NLP学习路径总结\\data\\english_data\\stopwords.txt')])
eng_stopwords=set(stopwords)


接下来要对数据做预处理，分为以下几个步骤：
1. 去掉html标签
1. 移除标点符号
1. 将句子切分成词
1. 去掉停用词
1. 重组为新的句子

In [4]:
def clean_text(text):
    text=BeautifulSoup(text,'html.parser').get_text()
    text=re.sub('[^a-zA-Z]',' ',text)
    words=text.lower().split()
    words=[w for w in words if w not in eng_stopwords]
    return ' '.join(words)

In [5]:
#将清洗的数据放到dataframe中
df['clean_review']=df.review.apply(clean_text)
df.head()

G:\anaconda3\envs\py37\lib\site-packages\bs4\__init__.py:439: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning


,id,sentiment,review,clean_review
0,5814_8,1,With all this stuff going down at the moment w...,stuff moment mj ve started listening music wat...
1,2381_9,1,"""The Classic War of the Worlds"" by Timothy Hin...",classic war worlds timothy hines entertaining ...
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,film starts manager nicholas bell giving inves...
3,3630_4,0,It must be assumed that those who praised this...,assumed praised film greatest filmed opera did...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,superbly trashy wondrously unpretentious explo...


In [7]:
from gensim.models.word2vec import Word2Vec

vectorizer=CountVectorizer(max_features=5000)
train_data_features=vectorizer.fit_transform(df.clean_review).toarray()
train_data_features.shape

 
# 设定词向量训练的参数
num_features = 300    # Word vector dimensionality
min_word_count = 40   # Minimum word count
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size
downsampling = 1e-3   # Downsample setting for frequent words

sentences = [['first', 'sentence'], ['second', 'sentence']]

model = Word2Vec(sentences=sentences, workers=num_workers,vector_size=num_features, min_count = min_word_count, window = context, sample = downsampling)
 
# If you don't plan to train the model any further, calling
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)
 
# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model.save(os.path.join('..', 'models', model))

RuntimeError: you must first build vocabulary before training the model

In [ ]:
#构建随机森林分类器并训练
forest=RandomForestClassifier(n_estimators=100)
forest=forest.fit(train_data_features,df.sentiment)
 
#删除不用的占内容变量
del df 
del train_data_features

In [ ]:
#读取测试数据并进行预测
datafile=os.path.join('F:\\NLP学习路径总结\\data\\english_data','testData.tsv')
df=pd.read_csv(datafile,sep='\t',escapechar='\\')
print('Number of reviews:{}'.format(len(df)))
df['clean_review']=df.review.apply(clean_text)
df.head()
 
test_data_features=vectorizer.transform(df.clean_review).toarray()
test_data_features.shape
 
result=forest.predict(test_data_features)
output=pd.DataFrame({'id':df.id,'sentiment':result})
output.head()

In [ ]:
# 最后一步，将预测结果写入csv文件

output.to_csv(os.path.join('F:\\NLP学习路径总结\\data\\english_data','Bag_of_Words_model.csv'),index=False)
 
 
del df
del test_data_features

# 最后一步，将预测结果写入csv文件

output.to_csv(os.path.join('F:\\NLP学习路径总结\\data\\english_data','Bag_of_Words_model.csv'),index=False)
 
 
del df
del test_data_features

In [12]:
def clean_text(text):
    text=BeautifulSoup(text,'html.parser').get_text()
    text=re.sub('[^a-zA-Z]',' ',text)
    words=text.lower().split()
    words=[w for w in words if w not in eng_stopwords]
    return ' '.join(words)

In [13]:
#将清洗的数据放到dataframe中
df['clean_review']=df.review.apply(clean_text)
df.head()

G:\anaconda3\envs\py37\lib\site-packages\bs4\__init__.py:439: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning


,id,sentiment,review,clean_review
0,5814_8,1,With all this stuff going down at the moment w...,stuff moment mj ve started listening music wat...
1,2381_9,1,"""The Classic War of the Worlds"" by Timothy Hin...",classic war worlds timothy hines entertaining ...
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,film starts manager nicholas bell giving inves...
3,3630_4,0,It must be assumed that those who praised this...,assumed praised film greatest filmed opera did...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,superbly trashy wondrously unpretentious explo...


In [14]:
from gensim.models.word2vec import Word2Vec

vectorizer=CountVectorizer(max_features=5000)
train_data_features=vectorizer.fit_transform(df.clean_review).toarray()
train_data_features.shape

 
# 设定词向量训练的参数
num_features = 300    # Word vector dimensionality
min_word_count = 40   # Minimum word count
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size
downsampling = 1e-3   # Downsample setting for frequent words
 
model = Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)
 
# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)
 
# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model.save(os.path.join('..', 'models', model_name))

NameError: name 'sentences' is not defined

In [ ]:
#构建随机森林分类器并训练
forest=RandomForestClassifier(n_estimators=100)
forest=forest.fit(train_data_features,df.sentiment)
 
#删除不用的占内容变量
del df 
del train_data_features

In [ ]:
#读取测试数据并进行预测
datafile=os.path.join('F:\\NLP学习路径总结\\data\\english_data','testData.tsv')
df=pd.read_csv(datafile,sep='\t',escapechar='\\')
print('Number of reviews:{}'.format(len(df)))
df['clean_review']=df.review.apply(clean_text)
df.head()
 
test_data_features=vectorizer.transform(df.clean_review).toarray()
test_data_features.shape
 
result=forest.predict(test_data_features)
output=pd.DataFrame({'id':df.id,'sentiment':result})
output.head()

In [ ]:
# 最后一步，将预测结果写入csv文件

output.to_csv(os.path.join('F:\\NLP学习路径总结\\data\\english_data','Bag_of_Words_model.csv'),index=False)
 
 
del df
del test_data_features